In [1]:
import random
import pandas as pd
import ydf
import wandb

Train a random forest model

Current best model num_trees=500 & l2_regularization=0.00

In [10]:
wandb.init(
    project = "stonks",
    config = {
        "num_trees": 100,
        "architecture": "GradientBoostedTrees",
    }
)

# Loop through each set of training and validation data
for i in range(13):  # Assuming i ranges from 0 to 12
    # Load training and validation data
    train_file = f"covset0/train_{i}.csv"
    valid_file = f"covset0/valid_{i}.csv"

    train_df = pd.read_csv(train_file)
    valid_df = pd.read_csv(valid_file)

    # Separate features and labels
    X_train = train_df.drop(columns=['DELTA_20_QUINTILES'])  # Replace with actual label column
    y_train = train_df['DELTA_20_QUINTILES']

    X_valid = valid_df.drop(columns=['DELTA_20_QUINTILES'])  # Replace with actual label column
    y_valid = valid_df['DELTA_20_QUINTILES']

    # Combine X and y for YDF
    train_data = pd.concat([X_train, y_train], axis=1)
    valid_data = pd.concat([X_valid, y_valid], axis=1)

    # Specify the label column for YDF
    label = "DELTA_20_QUINTILES"

    # Train the Gradient Boosted Trees model
    model = ydf.GradientBoostedTreesLearner(task=ydf.Task.CLASSIFICATION, 
                                    label=label,
                                    l2_regularization=0.00,
                                    num_trees=500,
                                    loss='MULTINOMIAL_LOG_LIKELIHOOD').train(train_data, valid=valid_df)


    # Evaluate the model
    evaluation = model.evaluate(valid_data)

    # Log the results with WandB
    wandb.log({
        'fold': i,
        'accuracy': evaluation.loss,  # Log cross-entropy or other metrics
    })

    print(f"Completed training and validation for fold {i}")

# Finalize WandB run
wandb.finish()

Train model on 252 training examples and 60 validation examples
Model trained in 0:00:00.318981
Completed training and validation for fold 0
Train model on 252 training examples and 60 validation examples
Model trained in 0:00:00.360069
Completed training and validation for fold 1
Train model on 252 training examples and 60 validation examples
Model trained in 0:00:00.323263
Completed training and validation for fold 2
Train model on 252 training examples and 60 validation examples
Model trained in 0:00:00.412015
Completed training and validation for fold 3
Train model on 252 training examples and 60 validation examples
Model trained in 0:00:00.368607
Completed training and validation for fold 4
Train model on 252 training examples and 60 validation examples
Model trained in 0:00:00.338097
Completed training and validation for fold 5
Train model on 252 training examples and 60 validation examples
Model trained in 0:00:00.382387
Completed training and validation for fold 6
Train model o

accuracy,██▁▁███▁▁▁▁▁▁
fold,▁▂▂▃▃▄▅▅▆▆▇▇█
accuracy,0.12128
fold,12


In [11]:
model.predict_proba(valid)

AttributeError: 'GradientBoostedTreesModel' object has no attribute 'predict_proba'

Train a Random Forest Model

In [ ]:
wandb.init(
    project = "stonks",
    config = {
        "num_trees": 100,
        "architecture": "GradientBoostedTrees",
    }
)

# Loop through each set of training and validation data
for i in range(13):  # Assuming i ranges from 0 to 12
    # Load training and validation data
    train_file = f"covset0/train_{i}.csv"
    valid_file = f"covset0/valid_{i}.csv"

    train_df = pd.read_csv(train_file)
    valid_df = pd.read_csv(valid_file)

    # Separate features and labels
    X_train = train_df.drop(columns=['DELTA_20_QUINTILES'])  # Replace with actual label column
    y_train = train_df['DELTA_20_QUINTILES']

    X_valid = valid_df.drop(columns=['DELTA_20_QUINTILES'])  # Replace with actual label column
    y_valid = valid_df['DELTA_20_QUINTILES']

    # Combine X and y for YDF
    train_data = pd.concat([X_train, y_train], axis=1)
    valid_data = pd.concat([X_valid, y_valid], axis=1)

    # Specify the label column for YDF
    label = "DELTA_20_QUINTILES"

    # Train the Gradient Boosted Trees model
    model = ydf.RandomForestLearner(task=ydf.Task.CLASSIFICATION,
                                    label=label,
                                    l2_regularization=0.01, loss='MULTINOMIAL_LOG_LIKELIHOOD').train(train_data, valid=valid_df)


    # Evaluate the model
    evaluation = model.evaluate(valid_data)

    # Log the results with WandB
    wandb.log({
        'fold': i,
        'accuracy': evaluation.loss,  # Log cross-entropy or other metrics
    })

    print(f"Completed training and validation for fold {i}")

# Finalize WandB run
wandb.finish()

In [ ]:
evaluation.loss

In [ ]:
model.evaluate(valid_data)